# **Web Scraping + Selenium**

Similar aos comandos do *pyautogui* e *pyperclip*. No entanto, há algumas vantagens:

        Selenium é capaz de continuar automatizando mesmo se a página web for reconfigurada;
        
        Selinium não exige tantos comandos manuais na montagem do código;
        
        Não há necessidade de usar o time.sleep()
        
----------

## **Desafio**

Para uma empresa importadora, deseja- se atualizar os preços dos produtos diariamente de acordo com a cotação do dólar, do euro e do ouro. 

        pegar na internet as cotações -> atualizar os preços na minha base de dados -> exportar a base para o sistema principal da empresa


**Instalação do webdrive*

~Cada computador apresenta um webdrive específico de acordo com o navegador padrão. 
Para o Chrome, o nome é chrome drive e devemos instalá-lo 

**Observação:** Tentar instalar o webdrive na mesma pasta onde está instalada o Pyhton. 

In [105]:
#Dentro da biblioteca selenium, eu desejo apenas o "livro" webdriver
#webdriver é uma classe especifica para conctar o webdrive do meu pc ao python

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


***O QUE SIGNIFICA FIND_ELEMENT?***
        
Uma função específica para encontrar um campo específico de um site. 

*By.XPATH* = Uma forma específica de pegar o link do campo. Existem outros, porém *By.XPATH* é o mais genérico 

*.send_keys(Chaves específicas para realizar comandos)

In [106]:
# Conectar o webdriver ao navegar do computador e entrar em um site específico

browser = webdriver.Chrome('chromedriver.exe')
browser.get("https://www.google.com/")
browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do dólar hoje")
browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
price_dolar = browser.find_element(By.XPATH, 
                                   '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#-------------------------

browser.get("https://www.google.com/")
browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do euro hoje")
browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
price_euro = browser.find_element(By.XPATH, 
                                   '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#------------------------

browser.get('https://www.melhorcambio.com/ouro-hoje')
price_gold =browser.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value').replace(',', '.')


<ipython-input-106-34235b3f6a05>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('chromedriver.exe')


In [107]:
print(f'Cotações\nDólar: R${price_dolar}\nEuro: R${price_euro}\nOuro: R${price_gold}')

Cotações
Dólar: R$5.523499999999999
Euro: R$6.3316377610000005
Ouro: R$323.18


In [108]:
# Próximo passo é importar a base de dados 

import pandas as pd 

In [109]:
frame = pd.read_excel(r'Produtos.xlsx')
display(frame)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


**frame.loc[linha,coluna]**

.loc indica para localizar exatamente aquelas posições na tabela. 

Porém podemos colocar opeações específicas dentro de linhas ou de colunas. 

        frame.loc[frame['Moeda'] == 'Dólar', Cotação]

Para toda linha cujo elemento na coluna moeda seja Dólar, modificaremos o elemento de mesma linha localizado na coluna 'Cotação'



In [110]:
# Atualização das cotações de maneira rápida e eficiente 

frame.loc[frame['Moeda'] == 'Dólar', 'Cotação'] = float(price_dolar)
frame.loc[frame['Moeda'] == 'Euro', 'Cotação'] = float(price_euro)
frame.loc[frame['Moeda'] == 'Ouro', 'Cotação'] = float(price_gold)

display(frame)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.523500,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.331638,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.523500,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.523500,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.331638,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.523500,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,323.180000,7000.00,1.15,8050.000


In [111]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Produtos         7 non-null      object 
 1   Preço Original   7 non-null      float64
 2   Moeda            7 non-null      object 
 3   Cotação          7 non-null      float64
 4   Preço de Compra  7 non-null      float64
 5   Margem           7 non-null      float64
 6   Preço de Venda   7 non-null      float64
dtypes: float64(5), object(2)
memory usage: 520.0+ bytes


In [112]:
# Atualização dos valores de preço de compra e preço de venda

frame['Preço de Compra'] = frame['Preço Original'] * frame['Cotação']
frame['Preço de Venda'] = frame['Preço de Compra'] * frame['Margem']

In [113]:
display(frame)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.523500,5523.444765,1.40,7732.822671
1,Carro Renault,4500.00,Euro,6.331638,28492.369925,2.00,56984.739849
2,Notebook Dell,899.99,Dólar,5.523500,4971.094765,1.70,8450.861101
3,IPhone,799.00,Dólar,5.523500,4413.276500,1.70,7502.570050
4,Carro Fiat,3000.00,Euro,6.331638,18994.913283,1.90,36090.335238
5,Celular Xiaomi,480.48,Dólar,5.523500,2653.931280,2.00,5307.862560
6,Joia 20g,20.00,Ouro,323.180000,6463.600000,1.15,7433.140000


In [114]:
# Arredondar todos os valores dentro de um DataFrame

for i in range(len(frame['Preço de Venda'])):
    frame.loc[i,'Preço de Venda']= round(frame.loc[i,'Preço de Venda'],3)

display(frame)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.523500,5523.444765,1.40,7732.823
1,Carro Renault,4500.00,Euro,6.331638,28492.369925,2.00,56984.740
2,Notebook Dell,899.99,Dólar,5.523500,4971.094765,1.70,8450.861
3,IPhone,799.00,Dólar,5.523500,4413.276500,1.70,7502.570
4,Carro Fiat,3000.00,Euro,6.331638,18994.913283,1.90,36090.335
5,Celular Xiaomi,480.48,Dólar,5.523500,2653.931280,2.00,5307.863
6,Joia 20g,20.00,Ouro,323.180000,6463.600000,1.15,7433.140


In [115]:
frame.to_excel('Preços_Produtos_atualizados.xlsx', index=False)